# Sequential Chats and Customer Onboarding

---

## Pre-requisites

- Create conda env using the following command from the root of this repo: `conda env create -f ./notebooks/multi-agent-systems/pyautogen/conda-env.yaml`.
- Please define the following environment variables in `.env` file:
    - `OPENAI_API_BASE="<ENDPOINT_URL>"`
    - `OPENAI_API_KEY="<API_KEY>"`

## Introduction

<img src="./images/cust-onboarding-overview.png" alt="Customer Onboarding Overview" style="width: 40%;"/>
<img src="./images/cust-onboarding-detailed.png" alt="Customer Onboarding Detailed" style="width: 40%;"/>

## Setup

In [15]:
import warnings
import os
from dotenv import load_dotenv, find_dotenv

warnings.filterwarnings('ignore')
_ = load_dotenv(find_dotenv())

Please find additional info about how to create LLM Configuration [over here](https://microsoft.github.io/autogen/docs/topics/llm_configuration).

In [16]:
llm_config = {
    "model": "gpt-4-turbo",
    "api_type": "azure",
    "base_url": os.environ.get("OPENAI_API_BASE"),
    "api_key": os.environ.get("OPENAI_API_KEY"),
}

## Creating the needed agents

In [30]:
from autogen import ConversableAgent

In [31]:
onboarding_personal_information_agent = ConversableAgent(
    name="Onboarding_Personal_Information_Agent",  # do not use spaces in agent name, throws error
    system_message='''You are a helpful customer onboarding agent,
    you are here to help new customers get started with our product.
    Your job is to gather customer's name and location.
    Do not ask for other information. Return 'TERMINATE' 
    when you have gathered all the information.''',
    llm_config=llm_config,
    code_execution_config=False,
    human_input_mode="NEVER",
)

In [32]:
onboarding_topic_preference_agent = ConversableAgent(
    name="Onboarding_Topic_preference_Agent",
    system_message='''You are a helpful customer onboarding agent,
    you are here to help new customers get started with our product.
    Your job is to gather customer's preferences on news topics.
    Do not ask for other information.
    Return 'TERMINATE' when you have gathered all the information.''',
    llm_config=llm_config,
    code_execution_config=False,
    human_input_mode="NEVER",
)

In [33]:
customer_engagement_agent = ConversableAgent(
    name="Customer_Engagement_Agent",
    system_message='''You are a helpful customer service agent
    here to provide fun for the customer based on the user's
    personal information and topic preferences.
    This could include fun facts, jokes, or interesting stories.
    Make sure to make it engaging and fun!
    Return 'TERMINATE' when you are done.''',
    llm_config=llm_config,
    code_execution_config=False,
    human_input_mode="NEVER",
    is_termination_msg=lambda msg: "terminate" in msg.get("content").lower(),
)

In [34]:
customer_proxy_agent = ConversableAgent(
    name="customer_proxy_agent",
    llm_config=False,
    code_execution_config=False,
    human_input_mode="ALWAYS",
    is_termination_msg=lambda msg: "terminate" in msg.get("content").lower(),
)

## Creating tasks

In [39]:
chats = [
    {
        "sender": onboarding_personal_information_agent,
        "recipient": customer_proxy_agent,
        "message": "Hello, I'm here to help you get started with our product. "
        "Could you tell me your name and location?",
        "summary_method": "reflection_with_llm",
        "summary_args": {
            "summary_prompt": "Return the customer information "
            "into as JSON object only: "
            "{'name': '', 'location': ''}",
        },
        "max_turns": 2,
        "clear_history": True,
    },
    {
        "sender": onboarding_topic_preference_agent,
        "recipient": customer_proxy_agent,
        "message": "Great! Could you tell me what topics you are "
        "interested in reading about?",
        "summary_method": "reflection_with_llm",
        "max_turns": 1,
        "clear_history": False,
    },
    {
        "sender": customer_proxy_agent,
        "recipient": customer_engagement_agent,
        "message": "Let's find something fun to read.",
        "max_turns": 1,
        "summary_method": "reflection_with_llm",
    },
]

## Start the onboarding process

In [40]:
from autogen import initiate_chats

chat_results = initiate_chats(chats)


********************************************************************************
Starting a new chat....

********************************************************************************
Onboarding_Personal_Information_Agent (to customer_proxy_agent):

Hello, I'm here to help you get started with our product. Could you tell me your name and location?

--------------------------------------------------------------------------------
customer_proxy_agent (to Onboarding_Personal_Information_Agent):

Andy

--------------------------------------------------------------------------------
Onboarding_Personal_Information_Agent (to customer_proxy_agent):

Thank you, Andy. Could you please provide your location as well?

--------------------------------------------------------------------------------
customer_proxy_agent (to Onboarding_Personal_Information_Agent):

Bangalore

--------------------------------------------------------------------------------

***************************************

## Print out the summary

In [43]:
for chat_result in chat_results:
    print(chat_result.summary)
    print("\n")

{
  "name": "Andy",
  "location": "Bangalore"
}


Andy, located in Bangalore, is interested in reading about dogs.


Andy from Bangalore is interested in reading about dogs, and a suggestion was made about the popularity of Labrador Retrievers in Bangalore, highlighting their roles as family pets and service dogs. Additionally, a light-hearted joke about a dog was shared to entertain Andy.




## Print out the cost

In [44]:
for chat_result in chat_results:
    print(chat_result.cost)
    print("\n")

{'usage_including_cached_inference': {'total_cost': 0.0063, 'gpt-4-turbo-2024-04-09': {'cost': 0.0063, 'prompt_tokens': 444, 'completion_tokens': 62, 'total_tokens': 506}}, 'usage_excluding_cached_inference': {'total_cost': 0}}


{'usage_including_cached_inference': {'total_cost': 0.0027700000000000003, 'gpt-4-turbo-2024-04-09': {'cost': 0.0027700000000000003, 'prompt_tokens': 214, 'completion_tokens': 21, 'total_tokens': 235}}, 'usage_excluding_cached_inference': {'total_cost': 0.0027700000000000003, 'gpt-4-turbo-2024-04-09': {'cost': 0.0027700000000000003, 'prompt_tokens': 214, 'completion_tokens': 21, 'total_tokens': 235}}}


{'usage_including_cached_inference': {'total_cost': 0.0161, 'gpt-4-turbo-2024-04-09': {'cost': 0.0161, 'prompt_tokens': 467, 'completion_tokens': 381, 'total_tokens': 848}}, 'usage_excluding_cached_inference': {'total_cost': 0.0161, 'gpt-4-turbo-2024-04-09': {'cost': 0.0161, 'prompt_tokens': 467, 'completion_tokens': 381, 'total_tokens': 848}}}


